<a href="https://colab.research.google.com/github/NoeliaDuranL/SIS420/blob/main/Laboratorio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LABORATORIO 3 - Duran Llaveta Noelia Paola - G2
Lo que se busca es predecir el tipo de violación de trafico que se ha cometido. Para eso se usa el dataset traffic violation que cuenta con 21 columnas de las cuales se usa 7 columnas sin contar la Y.

Link del dataset: https://www.kaggle.com/datasets/nikhil1e9/traffic-violations

Se usaron menos columnas de las esperadas puesto que el resto de datos no era relevante o los datos no se podian usar para la regresion logistica.

La Y tiene tres tipo: **warning(1), citation(2) y SERO(3)**

Después del procesamiento de datos se usa el dataframe **df_filtered**.

Al final del entrenamiento se obtiene un **57% de precisión**.

Link a github:

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

import pandas as pd

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [6]:
# Leer datos
df = pd.read_csv('/content/drive/MyDrive/SIS420/datasets/traffic_violations.csv', delimiter=',')

# 1. Procesamiento de datos

In [7]:
# Seleccionar solo las columnas requeridas
df_subset = df[['Belts', 'Personal.Injury', 'Property.Damage', 'Commercial.License', 'Commercial.Vehicle',
                'Year', 'Contributed.To.Accident', 'Gender', 'Violation.Type']]

# Mostrar las primeras filas del dataframe resultante
print(df_subset.head())

  Belts Personal.Injury Property.Damage Commercial.License Commercial.Vehicle  \
0    No              No              No                 No                 No   
1    No              No              No                 No                 No   
2    No              No              No                 No                 No   
3    No              No              No                 No                 No   
4    No              No              No                Yes                 No   

     Year Contributed.To.Accident Gender Violation.Type  
0  2013.0                      No      F       Citation  
1  2015.0                      No      M       Citation  
2  2000.0                      No      M       Citation  
3  2012.0                      No      M       Citation  
4  2010.0                      No      M       Citation  


In [8]:
# Obtener los tres tipos de violación con más ocurrencias
top_violations = df_subset['Violation.Type'].value_counts().head(3)

# Mostrar los resultados
print("Los tres tipos de violación con más ocurrencias son:")
print(top_violations)

Los tres tipos de violación con más ocurrencias son:
Warning     34382
Citation    32452
SERO         3506
Name: Violation.Type, dtype: int64


In [9]:
# Reemplazar 'no' con 0 y 'si' con 1 en la columna 'Belts'
df_subset['Belts'] = df_subset['Belts'].replace({'No': 0, 'Yes': 1})

# Mostrar las primeras filas del dataframe con la columna 'Belts' modificada
print(df_subset.head())


   Belts Personal.Injury Property.Damage Commercial.License  \
0      0              No              No                 No   
1      0              No              No                 No   
2      0              No              No                 No   
3      0              No              No                 No   
4      0              No              No                Yes   

  Commercial.Vehicle    Year Contributed.To.Accident Gender Violation.Type  
0                 No  2013.0                      No      F       Citation  
1                 No  2015.0                      No      M       Citation  
2                 No  2000.0                      No      M       Citation  
3                 No  2012.0                      No      M       Citation  
4                 No  2010.0                      No      M       Citation  


<ipython-input-9-25faa9d2c1ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Belts'] = df_subset['Belts'].replace({'No': 0, 'Yes': 1})


In [10]:
# Reemplazar valores en las columnas correspondientes
columns_to_replace = ['Personal.Injury', 'Property.Damage', 'Commercial.License', 'Commercial.Vehicle', 'Contributed.To.Accident']

for column in columns_to_replace:
    df_subset[column] = df_subset[column].replace({'No': 0, 'Yes': 1})

# Mostrar las primeras filas del dataframe con las columnas modificadas
print(df_subset.head())


<ipython-input-10-666a4796fba6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[column] = df_subset[column].replace({'No': 0, 'Yes': 1})


   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle    Year  Contributed.To.Accident Gender Violation.Type  
0                   0  2013.0                        0      F       Citation  
1                   0  2015.0                        0      M       Citation  
2                   0  2000.0                        0      M       Citation  
3                   0  2012.0                        0      M       Citation  
4                   0  2010.0                        0      M       Citation  


In [11]:
# Reemplazar en Gender, F=0 y M=1
df_subset['Gender'] = df_subset['Gender'].replace({'F': 0, 'M': 1})

# Mostrar las primeras filas del dataframe modificada
print(df_subset.head())


   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle    Year  Contributed.To.Accident Gender Violation.Type  
0                   0  2013.0                        0      0       Citation  
1                   0  2015.0                        0      1       Citation  
2                   0  2000.0                        0      1       Citation  
3                   0  2012.0                        0      1       Citation  
4                   0  2010.0                        0      1       Citation  


<ipython-input-11-84bc3cf58472>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Gender'] = df_subset['Gender'].replace({'F': 0, 'M': 1})


In [12]:
# Obtener la cantidad de datos de cada columna
cantidad_datos_por_columna = df_subset.count()

# Mostrar la cantidad de datos de cada columna
print("Cantidad de datos por columna:")
print(cantidad_datos_por_columna)


Cantidad de datos por columna:
Belts                      70340
Personal.Injury            70340
Property.Damage            70340
Commercial.License         70340
Commercial.Vehicle         70340
Year                       69906
Contributed.To.Accident    70340
Gender                     70340
Violation.Type             70340
dtype: int64


In [13]:
# Obtener la cantidad de valores nulos en cada columna
valores_nulos_por_columna = df_subset.isnull().sum()

# Mostrar la cantidad de valores nulos en cada columna
print("Cantidad de valores nulos por columna:")
print(valores_nulos_por_columna)


Cantidad de valores nulos por columna:
Belts                        0
Personal.Injury              0
Property.Damage              0
Commercial.License           0
Commercial.Vehicle           0
Year                       434
Contributed.To.Accident      0
Gender                       0
Violation.Type               0
dtype: int64


In [14]:
# Eliminar la columna 'Year' del dataframe
df_subset = df_subset.drop(columns=['Year'])

# Mostrar las primeras filas del dataframe actualizado
print(df_subset.head())


   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle  Contributed.To.Accident Gender Violation.Type  
0                   0                        0      0       Citation  
1                   0                        0      1       Citation  
2                   0                        0      1       Citation  
3                   0                        0      1       Citation  
4                   0                        0      1       Citation  


In [15]:
# Filtrar el dataframe para incluir solo las filas con los tres tipos de violación especificados
violation_types_to_keep = ['Warning', 'Citation', 'SERO']
df_filtered = df_subset[df_subset['Violation.Type'].isin(violation_types_to_keep)]

# Mostrar las primeras filas del dataframe filtrado
print(df_filtered.head())


   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle  Contributed.To.Accident Gender Violation.Type  
0                   0                        0      0       Citation  
1                   0                        0      1       Citation  
2                   0                        0      1       Citation  
3                   0                        0      1       Citation  
4                   0                        0      1       Citation  


In [16]:
# Obtener la cantidad de datos de cada columna en el dataframe filtrado
cantidad_datos_por_columna_filtrado = df_filtered.count()

# Mostrar la cantidad de datos de cada columna en el dataframe filtrado
print("Cantidad de datos por columna en el dataframe filtrado:")
print(cantidad_datos_por_columna_filtrado)


Cantidad de datos por columna en el dataframe filtrado:
Belts                      70340
Personal.Injury            70340
Property.Damage            70340
Commercial.License         70340
Commercial.Vehicle         70340
Contributed.To.Accident    70340
Gender                     70340
Violation.Type             70340
dtype: int64


In [17]:
# Obtener los valores únicos en la columna 'Violation.Type'
valores_unicos_violation_type = df_filtered['Violation.Type'].unique()

# Mostrar los valores únicos en la columna 'Violation.Type'
print("Valores únicos en la columna 'Violation.Type':")
print(valores_unicos_violation_type)


Valores únicos en la columna 'Violation.Type':
['Citation' 'SERO' 'Warning']


In [18]:
# Mapear los valores de 'Violation.Type' a valores numéricos
mapping = {'Warning': 1, 'Citation': 2, 'SERO': 3}
df_filtered['Violation.Type'] = df_filtered['Violation.Type'].replace(mapping)

# Mostrar las primeras filas del dataframe con los valores de 'Violation.Type' modificados
print(df_filtered.head())


   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle  Contributed.To.Accident Gender  Violation.Type  
0                   0                        0      0               2  
1                   0                        0      1               2  
2                   0                        0      1               2  
3                   0                        0      1               2  
4                   0                        0      1               2  


In [19]:
# Imprimir las primeras filas del dataframe
print("Primeras filas del dataframe:")
print(df_filtered.head())

# Verificar los tipos de datos en el dataframe
print("\nTipos de datos en el dataframe:")
print(df_filtered.dtypes)

# Verificar si hay valores nulos en el dataframe
print("\nValores nulos en el dataframe:")
print(df_filtered.isnull().sum())


Primeras filas del dataframe:
   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle  Contributed.To.Accident Gender  Violation.Type  
0                   0                        0      0               2  
1                   0                        0      1               2  
2                   0                        0      1               2  
3                   0                        0      1               2  
4                   0                        0      1               2  

Tipos de datos en el dataframe:
Belts                       int64
Personal.Injury             int64
Property.Damage             int64
Commer

In [20]:
# Verificar los valores únicos en la columna 'Gender'
print("Valores únicos en la columna 'Gender':")
print(df_filtered['Gender'].unique())


Valores únicos en la columna 'Gender':
[0 1 'U']


In [21]:
# Contar cuántos valores 'U' hay en la columna 'Gender'
count_u = (df_filtered['Gender'] == 'U').sum()
print("Cantidad de valores 'U' en la columna 'Gender':", count_u)


Cantidad de valores 'U' en la columna 'Gender': 90


In [22]:
# Calcular la moda de la columna 'Gender' excluyendo los valores 'U'
mode_gender = df_filtered[df_filtered['Gender'] != 'U']['Gender'].mode()[0]
print("Moda de la columna 'Gender' (excluyendo 'U'):", mode_gender)


Moda de la columna 'Gender' (excluyendo 'U'): 1


In [23]:
# Convertir 'U' a 1 en la columna 'Gender'
df_filtered['Gender'] = df_filtered['Gender'].replace({'U': 1})


In [24]:
# Imprimir las primeras filas del dataframe
print("Primeras filas del dataframe:")
print(df_filtered.head())

# Verificar los tipos de datos en el dataframe
print("\nTipos de datos en el dataframe:")
print(df_filtered.dtypes)

# Verificar si hay valores nulos en el dataframe
print("\nValores nulos en el dataframe:")
print(df_filtered.isnull().sum())


Primeras filas del dataframe:
   Belts  Personal.Injury  Property.Damage  Commercial.License  \
0      0                0                0                   0   
1      0                0                0                   0   
2      0                0                0                   0   
3      0                0                0                   0   
4      0                0                0                   1   

   Commercial.Vehicle  Contributed.To.Accident  Gender  Violation.Type  
0                   0                        0       0               2  
1                   0                        0       1               2  
2                   0                        0       1               2  
3                   0                        0       1               2  
4                   0                        0       1               2  

Tipos de datos en el dataframe:
Belts                      int64
Personal.Injury            int64
Property.Damage            int64
Com


# Clasificación multiclase

In [25]:
# La entrada es de 7 elemento contando con x0
input_layer_size  = 7

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 3

# columnas relevantes para X
features = ['Belts', 'Personal.Injury', 'Property.Damage', 'Commercial.License', 'Commercial.Vehicle', 'Contributed.To.Accident', 'Gender']
X = df_filtered[features].values

# Phising' para y
y = df_filtered['Violation.Type'].values

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
train_size = int(0.8 * len(df_filtered))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Asignar X_train y y_train a X y y respectivamente
X = X_train
y = y_train
m_train = len(y_train)

# Imprimir los datos de entrenamiento y prueba
print("Datos de entrenamiento:")
print(X_train)
print(y_train)
print("Datos de prueba:")
print(X_test)
print(y_test)


m = y.size

Datos de entrenamiento:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
[2 2 2 ... 1 1 1]
Datos de prueba:
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 1 1 ... 1 1 1]


In [26]:
# Imprimir la primera fila de características X y el vector de etiquetas y
print("Primera fila de características X:")
print(X[0, :])
print("Vector de etiquetas y:")
print(y)


Primera fila de características X:
[0 0 0 0 0 0 0]
Vector de etiquetas y:
[2 2 2 ... 1 1 1]


# Normalización de datos

In [27]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [28]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [29]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape

X = X_norm


# Vectorización de la función de costo

In [30]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [31]:
def lrCostFunction(theta, X, y, lambda_):

    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta


# Clasificacion One-vs-all


In [32]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [33]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(3, 8)


In [34]:
print(all_theta)

[[-1.24380805e+01  2.11849858e-15  2.08859577e-15  1.17485827e-15
   4.86422421e-16  3.28996475e-16  5.07928666e-15  1.41367903e-15]
 [-6.37866148e-01 -5.22099769e-02 -2.89228575e-01 -3.06762870e-01
  -2.68678633e-02  4.01663216e-02 -2.76492054e-01 -1.51601779e-01]
 [ 3.76941120e-01  5.49243859e-02  3.06798468e-01  3.36952225e-01
   2.01436774e-02 -5.43082610e-02  3.04176557e-01  1.41200567e-01]]


In [35]:
def predictOneVsAll(all_theta, X):


    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [36]:
# Imprime la forma de tus datos de entrenamiento
print("Forma de los datos de entrenamiento:", X_train.shape)

# Predice las etiquetas para tus datos de entrenamiento utilizando el clasificador entrenado
pred_train = predictOneVsAll(all_theta, X_train)

# Calcula la precisión del conjunto de entrenamiento comparando las predicciones con las etiquetas reales
accuracy_train = np.mean(pred_train == y_train) * 100
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(accuracy_train))

# Selecciona un subconjunto de tus datos de prueba
X_test_subset = X_test[10:150, :].copy()
print("Forma del subconjunto de datos de prueba:", X_test_subset.shape)

# Agrega una columna adicional para el término de sesgo a tus datos de prueba
X_test_subset = np.concatenate([np.ones((140, 1)), X_test_subset], axis=1)
print("Forma de los datos de prueba con sesgo:", X_test_subset.shape)

# Predice las etiquetas para el subconjunto de datos de prueba utilizando el clasificador entrenado
pred_test_subset = np.argmax(sigmoid(X_test_subset.dot(all_theta.T)), axis=1)
print("Predicciones para el subconjunto de datos de prueba:", pred_test_subset)

# Imprime las etiquetas reales del subconjunto de datos de prueba
print("Etiquetas reales del subconjunto de datos de prueba:", y_test[10:150])



Forma de los datos de entrenamiento: (56272, 7)
Precisión del conjunto de entrenamiento: 57.67%
Forma del subconjunto de datos de prueba: (140, 7)
Forma de los datos de prueba con sesgo: (140, 8)
Predicciones para el subconjunto de datos de prueba: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Etiquetas reales del subconjunto de datos de prueba: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
